# Leer datasets

In [1]:
import pandas as pd
import numpy as np

customer = pd.read_csv('../../data/real/customer_data.csv')
customer = customer.sample(n=1000, random_state=42)
# drop rows with NaN values
customer = customer.dropna()
customer.drop(columns=['id'], inplace=True)
#pasar a numerico el gender
customer['gender'] = customer['gender'].map({'Male': 0, 'Female': 1})
#pasar a numerico el education ['Bachelor' 'Masters' 'HighSchool' 'College']
education_mapping = {
    'HighSchool': 0,
    'College': 1,
    'Bachelor': 2,
    'Masters': 3
}
customer['education'] = customer['education'].map(education_mapping)
#pasar purchase_frequency a numerico
purchase_frequency_mapping = {
    'rare': 0,
    'occasional': 1,
    'frequent': 2,
}
customer['purchase_frequency'] = customer['purchase_frequency'].map(purchase_frequency_mapping)
#dumificar el region ['North', 'South', 'East', 'West']
customer = pd.get_dummies(customer, columns=['region'], prefix='region', drop_first=True, dtype=int)
#dumificar product_category
customer = pd.get_dummies(customer, columns=['product_category'], prefix='product', drop_first=True, dtype=int)
#dumificar loyalty_status
customer = pd.get_dummies(customer, columns=['loyalty_status'], prefix='loyalty', drop_first=True, dtype=int)
customer.head()

age  gender  income  education  purchase_frequency  purchase_amount  \
75721   19       0    6378          3                   0             3024   
80184   30       1   20232          1                   1             6721   
19864   26       0   11959          1                   1             4868   
76699   34       1   18059          1                   0             7386   
92991   31       0   12968          1                   0             5024   

       promotion_usage  satisfaction_score  region_North  region_South  \
75721                1                   5             0             1   
80184                0                   7             0             1   
19864                0                   6             1             0   
76699                0                   4             0             0   
92991                0                   6             1             0   

       region_West  product_Books  product_Clothing  product_Electronics  \
75721            0              1                 0                    0   
80184            0              0                 0                    1   
19864            0              0                 0                    1   
76699            1              0                 0                    1   
92991            0              1                 0                    0   

       product_Food  product_Health  product_Home  loyalty_Regular  \
75721             0               0             0                1   
80184             0               0             0                0   
19864             0               0             0                0   
76699             0               0             0                1   
92991             0               0             0                0   

       loyalty_Silver  
75721               0  
80184               1  
19864               1  
76699               0  
92991               1

# Realizamos los clusters

In [2]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, MeanShift, SpectralClustering, AffinityPropagation, Birch, OPTICS, BisectingKMeans
from sklearn.mixture import GaussianMixture
n_clusters = 3
clustering_algorithms = {
    'KMeans': KMeans(n_clusters=n_clusters, random_state=42),
    'DBSCAN': DBSCAN(eps=0.7, min_samples=5),
    'AgglomerativeClustering': AgglomerativeClustering(n_clusters=n_clusters),
}
results = {}
for name, algorithm in clustering_algorithms.items():
    try:
        algorithm.fit(customer)
        if hasattr(algorithm, 'labels_'):
            labels = algorithm.labels_
        elif hasattr(algorithm, 'predict'):
            labels = algorithm.predict(customer)
        else:
            labels = algorithm.means_.argmax(axis=0)  # For GaussianMixture
        results[name] = labels
    except Exception as e:
        print(f"Error with {name}: {e}")

In [3]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
customer_pca = pca.fit_transform(customer)
plt.figure(figsize=(15, 20))
for i, (name, labels) in enumerate(results.items(), 1):
    plt.subplot(4, 3, i)
    plt.scatter(customer_pca[:, 0], customer_pca[:, 1], c=labels, cmap='viridis', s=10)
    plt.title(name)

In [4]:
#definimos dunn index
def dunn_index(data, labels):
    unique_labels = np.unique(labels)
    if len(unique_labels) < 2:
        return 0.0

    inter_cluster_distances = []
    intra_cluster_distances = []

    for i in range(len(unique_labels)):
        for j in range(i + 1, len(unique_labels)):
            cluster_i = data[labels == unique_labels[i]]
            cluster_j = data[labels == unique_labels[j]]
            inter_distance = np.linalg.norm(cluster_i.mean(axis=0) - cluster_j.mean(axis=0))
            inter_cluster_distances.append(inter_distance)

        intra_distance = np.mean([np.linalg.norm(data[labels == unique_label] - data[labels == unique_label].mean(axis=0)) for unique_label in unique_labels])
        intra_cluster_distances.append(intra_distance)

    dunn_index_value = min(inter_cluster_distances) / max(intra_cluster_distances)
    return dunn_index_value

In [5]:
#podrias generar una metrica en que se calcule el valor maximo de distancia (entre las menores distancias de cada punto con otro dentro del mismo cluster) dividido por el valor minimo de distancia entre cualquier punto de un cluster con otro punto de otro cluster
import numpy as np
from sklearn.metrics import pairwise_distances

def custom_metric(X, labels):
    X = np.array(X)
    distances = pairwise_distances(X)
    n = len(X)
    
    # Numerador: máximo de las mínimas distancias dentro del mismo cluster
    max_min_intra = -np.inf
    for i in range(n):
        same_cluster = (labels == labels[i])
        same_cluster[i] = False  # Exclude itself
        if np.any(same_cluster):
            min_dist = np.min(distances[i][same_cluster])
            max_min_intra = max(max_min_intra, min_dist)
    
    # Denominador: mínima distancia entre puntos de distintos clusters
    min_inter = np.inf
    for i in range(n):
        other_cluster = (labels != labels[i])
        if np.any(other_cluster):
            min_dist = np.min(distances[i][other_cluster])
            min_inter = min(min_inter, min_dist)

    M = max_min_intra / min_inter if min_inter != 0 else np.inf
    return 1 /( 1 + M)


In [6]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from permetrics import ClusteringMetric
def evaluate_clustering(data, labels):
    if len(set(labels)) < 2:
        return {
            'Silhouette Score': None,
            'Calinski-Harabasz Score': None,
            'Davies-Bouldin Score': None,
            'Dunn Index': None,
            'custom_metric': None,
            'dbcv': None
        }
    
    silhouette = silhouette_score(data, labels)
    calinski_harabasz = calinski_harabasz_score(data, labels)
    davies_bouldin = davies_bouldin_score(data, labels)
    dunn = dunn_index(data.values, labels)
    custom = custom_metric(data.values, labels)
    dbcv = ClusteringMetric(X=data.values, y_pred=labels).DBCVI()

    return {
        'Silhouette Score': silhouette,
        'Calinski-Harabasz Score': calinski_harabasz,
        'Davies-Bouldin Score': davies_bouldin,
        'Dunn Index': dunn,
        'custom_metric': custom,
        'dbcv': dbcv
    }
evaluation_results = {}
for name, labels in results.items():
    evaluation_results[name] = evaluate_clustering(customer, labels)
evaluation_df = pd.DataFrame(evaluation_results).T
evaluation_df = evaluation_df.rename(columns={
    'Silhouette Score': 'Silhouette',
    'Calinski-Harabasz Score': 'Calinski-Harabasz',
    'Davies-Bouldin Score': 'Davies-Bouldin',
    'Dunn Index': 'Dunn',
    'custom_metric': 'Custom Metric'
})
evaluation_df

Silhouette Calinski-Harabasz Davies-Bouldin      Dunn  \
KMeans                    0.555753       3417.362505       0.550635  0.172665   
DBSCAN                        None              None           None      None   
AgglomerativeClustering   0.537102       3051.902276       0.558039  0.164443   

                        Custom Metric      dbcv  
KMeans                       0.093143  0.868604  
DBSCAN                           None      None  
AgglomerativeClustering      0.106866  0.878138

In [7]:
#elige los mejores 3 resultados ponderando el orden de cada métrica
def weighted_score(row):
    weights = {
        'Silhouette': 0.2,
        'Calinski-Harabasz': -0.2,
        'Davies-Bouldin': 0.2,  # Negativo porque queremos minimizar
        'Dunn': 0.2,
        'Custom Metric': 0.5 # Puedes ajustar este peso según su importancia
    }
    score = (row['Silhouette'] * weights['Silhouette'] +
                1/(1+row['Calinski-Harabasz']) * weights['Calinski-Harabasz'] +
                1/(1+row['Davies-Bouldin']) * weights['Davies-Bouldin'] +
                row['Dunn'] * weights['Dunn'] +
                row['Custom Metric'] * weights['Custom Metric'])
    return score
# Replace None values with 0 for scoring
evaluation_df_filled = evaluation_df.fillna(0)
evaluation_df_filled['Weighted Score'] = evaluation_df_filled.apply(weighted_score, axis=1)
top_3_algorithms = evaluation_df_filled.nlargest(3, 'Weighted Score')
top_3_algorithms

/tmp/ipykernel_274005/2114172724.py:17: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  evaluation_df_filled = evaluation_df.fillna(0)


Silhouette  Calinski-Harabasz  Davies-Bouldin  \
AgglomerativeClustering    0.537102        3051.902276        0.558039   
KMeans                     0.555753        3417.362505        0.550635   
DBSCAN                     0.000000           0.000000        0.000000   

                             Dunn  Custom Metric      dbcv  Weighted Score  
AgglomerativeClustering  0.164443       0.106866  0.878138        0.322043  
KMeans                   0.172665       0.093143  0.868604        0.321176  
DBSCAN                   0.000000       0.000000  0.000000        0.000000

# Interpretar

In [8]:
import sys
sys.path.append('../..')
from ripper import MultiClassRipperClassifier
from dsgd import DSClassifierMultiQ
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree as sk_tree

In [9]:
# interpretamos solo los mejores 3 resultados y guardamos los modelos
saved_models = {}
top_3_labels = {name: results[name] for name in top_3_algorithms.index}
top_3_data = customer.copy()
for name, labels in top_3_labels.items():
    n = len(np.unique(labels))
    X_train, X_test, y_train, y_test = train_test_split(top_3_data, labels, test_size=0.2, random_state=42)
    print(f'Processing {name} with {n} clusters')
    
    # Ripper
    ripper = MultiClassRipperClassifier(random_state=43)
    ripper.fit(X_train, y_train)
    y_pred_ripper = ripper.predict(X_test)
    print(f'Ripper Accuracy for {name}: {accuracy_score(y_test, y_pred_ripper):.2f}')
    saved_models.setdefault(name, {})['ripper'] = ripper
    
    # Decision Tree
    dtree = DecisionTreeClassifier(random_state=42, max_depth=5)
    dtree.fit(X_train, y_train)
    dtree_pred = dtree.predict(X_test)
    print(f'Decision Tree Accuracy for {name}: {accuracy_score(y_test, dtree_pred):.2f}')
    saved_models[name]['dtree'] = dtree
    
    # DSGD
    ds = DSClassifierMultiQ(n, min_iter=20, max_iter=200, debug_mode=True, lossfn="MSE", num_workers=0, min_dloss=1e-7)
    X = X_train.to_numpy()
    losses, epoch, dt = ds.fit(X, y_train, add_single_rules=True, single_rules_breaks=3, add_mult_rules=False, column_names=X_train.columns)
    x_test = X_test.to_numpy()
    y_pred_ds = ds.predict(x_test)
    print(f'DSGD Accuracy for {name}: {accuracy_score(y_test, y_pred_ds):.2f}')
    saved_models[name]['ds'] = ds
    
    print('--' * 40)


Processing AgglomerativeClustering with 3 clusters
Ripper Accuracy for AgglomerativeClustering: 0.93
Decision Tree Accuracy for AgglomerativeClustering: 1.00
Optimization started

Training time: 39.34s, epochs: 200

Least training loss reached: 0.052
DSGD Accuracy for AgglomerativeClustering: 0.92
--------------------------------------------------------------------------------
Processing KMeans with 3 clusters
Ripper Accuracy for KMeans: 0.92
Decision Tree Accuracy for KMeans: 0.97
Optimization started

Training time: 39.54s, epochs: 200

Least training loss reached: 0.033
DSGD Accuracy for KMeans: 0.92
--------------------------------------------------------------------------------
Processing DBSCAN with 1 clusters
Ripper Accuracy for DBSCAN: 1.00
Decision Tree Accuracy for DBSCAN: 1.00
Optimization started


preprocess.py.py: ._get_pos_class: RuntimeWarning: 
No negative samples. All target labels=1.

ripper.py: .fit: RuntimeWarning: 
No negative samples. Existing target labels=[1].

ripper.py: .fit | base.py: ._check_allpos_allneg: RuntimeWarning: 
Ruleset is empty. All predictions it makes with method .predict will be negative. It may be untrained or was trained on a dataset split lacking positive examples.



RuntimeError: Class values must be non-negative.